In [1]:
import numpy as np
import os

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

import sMZI as itf

In [2]:
def print_matrix(M: np.ndarray, prec: int=2):
    """
    Function to print a given matrix in a nice way.
    
    Parameters
    ------
    M : matrix to print
    prec : floating point precision
    """
    for row in M:
        print(f"{np.array2string(row, precision=prec ,formatter={'float': lambda row: f'{row:.2f}'},separator=', ')}")
    print('\n')

def print_data_ext_ps(V1: np.complex_, V2: np.complex_):
    """
    Function to print data on the effect of the external phaseshift
    that's used to match the phases of two given elements.
    
    Parameters
    ------
    V1 : element of the auxillary matrix `V`
    V2 : subsequent element of **V1**
    
    ---
    
    Further description:
    ---
    When it comes to the effect of the external phaseshift  `P`:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x,y+1]``
    
    ---
    When it comes to the effect of `exp(i*summa)` in `M`:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x-1,y-1]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x+1,y+1]`` ,  ``V2 = V[x,y+1]``
    
    """
    
    eq_stat = ((np.angle(V1) - np.angle(V2)).round(10) == 0)
    eq_str = "\nThey are matching!"
    neq_str = "\nThey are NOT matching!\n"

    print('Affected elements:\t',
        "{:.2f}, {:.2f}".format(V1, V2))
    print('Corresponding angles:\t',
        "{:.2f}, {:.2f}".format(np.angle(V1), np.angle(V2)))

    if eq_stat:
        print(eq_str)
    else:
        print(neq_str)
        print('Full length of angles:\n',
            "{}, {}".format(np.angle(V1), np.angle(V2)))
        print('\nTheir difference:\t',
            "{}".format(np.angle(V1) - np.angle(V2)))

In [3]:
U = itf.random_unitary(4)

# Test of algorithm steps

## Odd diagonals

### Finding external phaseshifts
They match the given elements' phases

In [137]:
V = np.conjugate(U.T)
# odd diags: 1,3,5...
m = U.shape[0]
j = 1

x = m-1
y = j-1
s = y+1

P = itf.external_ps(m, s, V[x,y], V[x,y+1])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,P)
# print("NOTE: {}. column was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x,y+1])

Affected elements:	 0.51-0.13j, 0.58-0.14j
Corresponding angles:	 -0.24, -0.24

They are matching!


### Finding internal phaseshifts
$\delta$ and $\sum$

In [138]:
# looking at odd diagonal version!

# minus sign ! Note that the function custom_arctan(V1,V2) computes the arctan of -V1/V2 !
delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = 0

#compute the angles theta1 and theta2:
theta1 = delta-summ
theta2 = 2*summ-theta1
print('Check internal phases')
print((theta1+theta2)/2-summ)
print((theta1-theta2)/2-delta)

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
print_matrix(V)

delta:	-0.85-0.00j	 -> real: False
angle:	-180.0°
Check internal phases
0j
0j

Block of M:

[1.29e-17-0.75j, 1.07e-16+0.66j]
[ 1.07e-16+0.66j, -1.29e-17+0.75j]


[ 0.09+0.61j,  0.13-0.39j, -0.38+0.44j,  0.34-0.05j]
[ 0.01+0.69j, -0.12+0.1j ,  0.55-0.38j,  0.07+0.22j]
[-0.38+0.04j, -0.41-0.04j, -0.09-0.29j,  0.33-0.69j]
[-1.39e-17-9.09e-17j,  1.92e-01+7.73e-01j,  1.02e-01+3.37e-01j,
  4.86e-01-7.77e-02j]




In [139]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(-0.00) Im(-0.00)
abs: 0.00, angle: -98.72°



## Even diagonals

### Finding external phaseshifts
They match the given elements' phases

In [140]:
# V = np.conjugate(U.T)
# even diags: 2,4,6...
m = U.shape[0]
j = 2

x = m-j
y = 0
s = x-1

P = itf.external_ps(m, s, V[x,y], V[x-1,y])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(P,V)
# print("NOTE: {}. row was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x-1,y])

Affected elements:	 -0.38+0.04j, -0.69+0.06j
Corresponding angles:	 3.05, 3.05

They are matching!


In [121]:
#My ansatz

# V = np.conjugate(U.T)
# even diags: 2,4,6...
m = U.shape[0]
j = 2

x = m-j
y = 0

P = itf.external_ps(m, 0, V[x,y], V[x-1,y]) #ToDo: Find out the logic where to put the external PS!
# print('old V:\n')
print_matrix(P)
V = np.matmul(P,V)
# print("NOTE: {}. row was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x-1,y])

[1.-2.22e-16j, 0.+0.00e+00j, 0.+0.00e+00j, 0.+0.00e+00j]
[0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]
[0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]
[0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]


Affected elements:	 0.51-0.50j, 0.18-0.18j
Corresponding angles:	 -0.78, -0.78

They are matching!


### Finding internal phaseshifts
$\delta$ and $\sum$

In [122]:
x

2

In [141]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(-V[x-1,y], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = np.angle(V[x+1,y+1]) - np.angle(V[x-1,y+1]*np.cos(delta) - V[x,y+1]*np.sin(delta)) + np.pi/2

# print('\nangles to get equal by summa:\n',
#       np.angle(V[x+1,y+1]),
#       np.angle(V[x,y+1]*np.cos(delta) - V[x+1,y+1]*np.sin(delta)),'\n',
#       'summa:',summ)

# summ = 0

# might need to change into 'k' dependence
modes = [x-1, x]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

# print('\nBlock of M:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])
# print_matrix(M)


# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	1.06+0.00j	 -> real: False
angle:	1.3730548962059526e-15°


In [142]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
print('Elements affected by e^(i*summ):\n')
print_data_ext_ps(V[x+1,y+1], V[x,y+1])

Nulled element:	Re(-0.00) Im(-0.00)
abs: 0.00, angle: -106.54°

Elements affected by e^(i*summ):

Affected elements:	 0.19+0.77j, -0.07-0.30j
Corresponding angles:	 1.33, -1.81

They are NOT matching!

Full length of angles:
 1.3268909024409978, -1.8147017511487955

Their difference:	 3.141592653589793


### Continuation on the chosen diagonal
Without correct $\sum$ it's wrong

In [143]:
x += 1
y += 1

delta = itf.custom_arctan(-V[x-1,y], V[x,y])
print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = 0

modes = [x-1, x]     # initial mode-pairs
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

# print('\nBlock of M:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])

# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(x,x+1))
# print('new V:\n')
# print_matrix(V)

delta:	-0.37+0.00j	 -> real: False
angle:	180.0°


In [144]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(-0.00) Im(0.00)
abs: 0.00, angle: 163.15°



In [145]:
print_matrix(V.round(10))

[ 0.09+0.61j,  0.13-0.39j, -0.38+0.44j,  0.34-0.05j]
[ 0.21+0.76j, -0.06+0.31j,  0.25-0.37j, -0.26+0.07j]
[ 0.  -0.j  , -0.83+0.21j, -0.44-0.07j,  0.17+0.2j ]
[ 0.  -0.j  , -0.  +0.j  ,  0.22+0.46j, -0.23+0.83j]




## Odd diagonal 2.
Diagonal with more than one element

### Finding external phaseshifts
They match the given elements' phases

In [146]:
# odd diags: 1,3,5...
m = U.shape[0]
j = 3

x = m-1
y = j-1
s = y+1

P = itf.external_ps(m, s, V[x,y], V[x,y+1])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,P)
# print("NOTE: {}. column was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x,y+1])
print_matrix(V)

Affected elements:	 0.22+0.46j, 0.37+0.78j
Corresponding angles:	 1.13, 1.13

They are matching!
[ 0.09+0.61j,  0.13-0.39j, -0.38+0.44j,  0.22-0.26j]
[ 0.21+0.76j, -0.06+0.31j,  0.25-0.37j, -0.15+0.22j]
[ 4.91e-17-2.42e-18j, -8.28e-01+2.06e-01j, -4.45e-01-6.68e-02j,
  2.62e-01+3.94e-02j]
[ 1.26e-16-3.26e-17j, -4.91e-17+1.49e-17j,  2.19e-01+4.58e-01j,
  3.71e-01+7.77e-01j]




### Finding internal phaseshifts
$\delta$ and $\sum$

In [147]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = np.angle(V[x-1,y-1]) - np.angle(V[x-1,y]*np.sin(delta) + V[x-1,y+1]*np.cos(delta)) + np.pi/2

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])
print_matrix(M)

# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
print(np.angle(V[2][1]),np.angle(V[2][2]))

delta:	-1.04+0.00j	 -> real: False
angle:	180.0°

Block of M:

[-0.8 +0.33j,  0.47-0.19j]
[0.47-0.19j, 0.8 -0.33j]


[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]
[0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]
[ 0.  +0.j  ,  0.  +0.j  , -0.8 +0.33j,  0.47-0.19j]
[0.  +0.j  , 0.  +0.j  , 0.47-0.19j, 0.8 -0.33j]


2.8976872292358946 -0.24390542435389884


In [148]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(-0.00) Im(-0.00)
abs: 0.00, angle: -114.06°



In [149]:
x -= 1
y -= 1

delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = np.angle(V[x-1,y-1]) - np.angle(V[x-1,y]*np.sin(delta) + V[x-1,y+1]*np.cos(delta)) + np.pi/2

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
print_matrix(V)
print(np.angle(V[1][0]),np.angle(V[1][1]))

delta:	0.55-0.00j	 -> real: True
angle:	-0.0°

Block of M:

[-0.47+0.23j, -0.76+0.38j]
[-0.76+0.38j,  0.47-0.23j]


[9.22e-02+6.06e-01j, 1.19e-01+7.81e-01j, 7.05e-17-1.72e-16j,
 6.62e-17-1.62e-16j]
[ 2.05e-01+7.63e-01j, -1.59e-01-5.92e-01j,  2.10e-16+7.15e-17j,
 -1.52e-16+2.53e-16j]
[ 4.91e-17-2.42e-18j,  4.16e-17-9.65e-18j,  7.59e-01-6.52e-01j,
 -8.36e-17+3.28e-17j]
[ 1.26e-16-3.26e-17j,  1.36e-16+7.66e-17j, -3.94e-17-8.83e-17j,
  7.44e-01+6.69e-01j]


1.30785435631208 -1.8337382972777136


In [150]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(0.00) Im(-0.00)
abs: 0.00, angle: -13.06°



In [151]:
x -= 1
y -= 1

delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
#summ = np.angle(V[x-1,y-1]) - np.angle(V[x-1,y]*np.sin(delta) + V[x-1,y+1]*np.cos(delta))
summ = 0

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	0.66-0.00j	 -> real: False
angle:	-1.1020230613341204e-14°

Block of M:

[ 1.38e-16+0.61j, -2.95e-17+0.79j]
[-2.95e-17+0.79j, -1.38e-16-0.61j]




In [152]:
print(x,y)
V.round(3)

1 0


array([[-0.989+0.15j ,  0.   -0.j   ,  0.   -0.j   ,  0.   -0.j   ],
       [-0.   +0.j   , -0.966+0.26j ,  0.   +0.j   , -0.   +0.j   ],
       [ 0.   +0.j   , -0.   +0.j   ,  0.759-0.652j, -0.   +0.j   ],
       [-0.   +0.j   ,  0.   +0.j   , -0.   -0.j   ,  0.744+0.669j]])

now we implement the other external phases which are shifted to the middle

In [153]:
m = 4
for j in range(2,m+1):
   # xi = np.angle(V[0][0])-np.angle(V[j-1][j-1])
    V = np.dot(V,itf.external_ps(m, j-1, V[0,0], V[j-1,j-1]))
print_matrix(V.round(10))

[-0.99+0.15j,  0.  -0.j  , -0.  +0.j  ,  0.  +0.j  ]
[-0.  +0.j  , -0.99+0.15j, -0.  -0.j  , -0.  -0.j  ]
[ 0.  +0.j  , -0.  +0.j  , -0.99+0.15j,  0.  -0.j  ]
[-0.  +0.j  ,  0.  +0.j  , -0.  +0.j  , -0.99+0.15j]




# Test of module
## <center>TODO</center>
* check if $V$ is a diagonal matrix after the decomposition $\rightarrow$ done
---
- extend module to save found phases: $\phi$, $\delta$, $\sum$ $\Rightarrow$ $\theta_1$ & $\theta_2$ $\rightarrow$ done, but maybe needs to be adapted
- check the function in module that recreates the initial unitary matrix based on the decomposed matrices (probably needs work)
- update the drawing function in the module such that it follows the updated **Clement's** `draw()` function but with sMZI:s
- Finally: sweep through the whole module and finalise *comments*, *documentation*, *variables* and implement possible *optimalisations*
---
* optional: `import numba` for faster execution; optimized machine code at runtime

In [7]:
# including imports again in case someone only wishes
# to run this part of the notebook
import numpy as np
import os

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

import sMZI as itf

def print_matrix(M: np.ndarray, prec: int=2):
    """
    Function to print a given matrix in a nice way.
    
    Parameters
    ------
    M : matrix to print
    prec : floating point precision
    """
    for row in M:
        print(f"{np.array2string(row, precision=prec ,formatter={'float': lambda row: f'{row:.2f}'},separator=', ', suppress_small=True)}")
    print('\n')

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)


U = itf.random_unitary(5)

In [8]:
final_matrix = itf.square_decomposition(U)

In [9]:
# print_matrix(np.conjugate(final_matrix.T))
print_matrix(final_matrix)

TypeError: 'Interferometer' object is not iterable

$$\uparrow$$
### <center> The above is only zeroed in the lower triangular part! </center>